In [1]:
from mxnet import autograd, nd
from mxnet.gluon import nn

In [2]:
# 2d卷积操作
def corr2d(X, K):
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0, 1], [2, 3]])
corr2d(X, K)


[[ 19.  25.]
 [ 37.  43.]]
<NDArray 2x2 @cpu(0)>

In [4]:
# 二维卷积层
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))
    
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

In [5]:
# 图像中的物体边缘
# 构造一个中间4列为黑的图像
X = nd.ones((6, 8))
X[:, 2:6] = 0
X


[[ 1.  1.  0.  0.  0.  0.  1.  1.]
 [ 1.  1.  0.  0.  0.  0.  1.  1.]
 [ 1.  1.  0.  0.  0.  0.  1.  1.]
 [ 1.  1.  0.  0.  0.  0.  1.  1.]
 [ 1.  1.  0.  0.  0.  0.  1.  1.]
 [ 1.  1.  0.  0.  0.  0.  1.  1.]]
<NDArray 6x8 @cpu(0)>

In [6]:
# 高和宽分别为1和2的卷积核
K = nd.array([[1, -1]])

In [7]:
Y = corr2d(X, K)

In [8]:
# 卷积层可以通过重复使用卷积核有效的表征局部空间
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [9]:
# 通过数据学习核数组
# 构造一个输出通道数为1，核数组是(1, 2)的二维卷积层
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

In [10]:
# ⼆维卷积层使⽤ 4 维输⼊输出，格式为（样本，通道，⾼，宽），这⾥批量⼤⼩（批量中的样本数）
# 通道数均为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
    l.backward()
    # 为了简单起见这里忽略了偏差
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 4.949
batch 4, loss 0.831
batch 6, loss 0.140
batch 8, loss 0.024
batch 10, loss 0.004


In [11]:
conv2d.weight.data().reshape((1, 2))


[[ 0.98949999 -0.98737049]]
<NDArray 1x2 @cpu(0)>